In [1]:
! pip install tensorflow-gpu


  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      Traceback (most recent call last):
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer)
        File "/opt/conda/lib/python3.10/site-packages/setuptools/_vendor/packaging/_parser.py", line 126, in _parse_requirement_details
          marker = _parse_requirement_marker(
        File 

In [2]:
import tensorflow as tf

# Check for GPU availability
if tf.test.is_gpu_available():
    print("GPU is available and will be used for training.")
else:
    print("GPU is not available.")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


GPU is available and will be used for training.


In [3]:
# Explicitly specify GPU device
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Set memory growth to prevent GPU memory allocation from the beginning
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    # Set to use the first GPU (device 0)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')


In [4]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Flatten, Dense, Dropout

In [5]:
# Function to load and preprocess data
def load_and_preprocess_data(csv_file):
    data = pd.read_csv(csv_file)
    input_column = 'ii'
    target_column = 'i'

    input_data = data[input_column].values
    target_data = data[target_column].values

    scaler = MinMaxScaler()
    input_data = scaler.fit_transform(input_data.reshape(-1, 1))
    target_data = scaler.fit_transform(target_data.reshape(-1, 1))

    sequence_length = 1000

    X, y = [], []
    for i in range(len(input_data) - sequence_length):
        X.append(input_data[i:i+sequence_length])
        y.append(target_data[i+sequence_length])

    X = np.array(X)
    y = np.array(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

In [6]:
# Initialize the model
model = Sequential()
model.add(Conv1D(128, 5, activation='relu', input_shape=(1000, 1)))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2))
model.add(LSTM(64))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')


In [7]:
folder_path = '/kaggle/input/ecg-preprocessed/preprocessed'

In [8]:
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]


In [9]:
for csv_file in csv_files:
    X_train, X_test, y_train, y_test = load_and_preprocess_data(os.path.join(folder_path, csv_file))
    model.fit(X_train, y_train, epochs=10, batch_size=512)

Epoch 1/10
179/179 [==============================] - 23s 66ms/step - loss: 0.0097
Epoch 2/10
179/179 [==============================] - 12s 64ms/step - loss: 7.1610e-04
Epoch 3/10
179/179 [==============================] - 11s 64ms/step - loss: 6.4117e-04
Epoch 4/10
179/179 [==============================] - 11s 64ms/step - loss: 5.8960e-04
Epoch 5/10
179/179 [==============================] - 11s 64ms/step - loss: 5.9471e-04
Epoch 6/10
179/179 [==============================] - 11s 64ms/step - loss: 5.6762e-04
Epoch 7/10
179/179 [==============================] - 11s 64ms/step - loss: 5.4411e-04
Epoch 8/10
179/179 [==============================] - 11s 64ms/step - loss: 5.4522e-04
Epoch 9/10
179/179 [==============================] - 11s 64ms/step - loss: 5.3149e-04
Epoch 10/10
179/179 [==============================] - 11s 64ms/step - loss: 5.2813e-04
Epoch 1/10
179/179 [==============================] - 12s 64ms/step - loss: 0.0032
Epoch 2/10
179/179 [==============================

In [12]:
model.save('lead_I.h5')